In [33]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import time

# create hex_color_picker

def hex_to_bgr(hex_color):
    hex_color = hex_color.lstrip('#')
    lv = len(hex_color)
    return tuple(int(hex_color[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))[::-1]

def reference_frame(frame, mask_bound, hex_color_1, hex_color_2, threshold = 40):
    
    x1, y1, x2, y2 = mask_bound
    roi = frame[y1:y2, x1:x2]

    # black
    bgr_color_1 = hex_to_bgr(hex_color_1) # "#C8CE7B"
    lower_bound_1 = np.array([max(0, bgr_color_1[0] - threshold), max(0, bgr_color_1[1] - threshold), max(0, bgr_color_1[2] - threshold)])
    upper_bound_1 = np.array([min(255, bgr_color_1[0] + threshold), min(255, bgr_color_1[1] + threshold), min(255, bgr_color_1[2] + threshold)])
    mask_1 = cv2.inRange(roi, lower_bound_1, upper_bound_1)

    y_coord_1, x_coord_1 = np.where(mask_1 != 0)
    roi[mask_1 != 0] = [255, 0, 0]
    coord_array_1 = np.stack((y_coord_1, x_coord_1), axis=-1)
    sorted_array_1 = coord_array_1[coord_array_1[:, 1].argsort()]
    

    # white
    bgr_color_2 = hex_to_bgr(hex_color_2) # "#C8CE7B"
    lower_bound_2 = np.array([max(0, bgr_color_2[0] - threshold), max(0, bgr_color_2[1] - threshold), max(0, bgr_color_2[2] - threshold)])
    upper_bound_2 = np.array([min(255, bgr_color_2[0] + threshold), min(255, bgr_color_2[1] + threshold), min(255, bgr_color_2[2] + threshold)])
    mask_2 = cv2.inRange(roi, lower_bound_2, upper_bound_2)

    y_coord_2, x_coord_2 = np.where(mask_2 != 0)
    roi[mask_2 != 0] = [0, 0, 255]
    coord_array_2 = np.stack((y_coord_2, x_coord_2), axis=-1)
    sorted_array_2 = coord_array_2[coord_array_2[:, 1].argsort()]

    # DBSCAN Black
    if sorted_array_1.size > 0:
        dbscan_1 = DBSCAN(eps=5, min_samples=10)  # Adjust eps and min_samples as needed
        clusters_1 = dbscan_1.fit_predict(sorted_array_1)
    
        # Organize points into a dictionary based on their cluster
        cluster_dict_1 = {}
        for point, cluster_idx in zip(sorted_array_1, clusters_1):
            if cluster_idx != -1:  # Filter out noise points if needed
                cluster_dict_1.setdefault(cluster_idx, []).append(point.tolist())
    else:
        print("No blue points found in the image.")
        cluster_dict_1 = {}

    for cluster_idx in cluster_dict_1:
        cluster_dict_1[cluster_idx] = np.array(cluster_dict_1[cluster_idx])

    # DBSCAN White
    if sorted_array_2.size > 0:
        dbscan_2 = DBSCAN(eps=5, min_samples=10)  # Adjust eps and min_samples as needed
        clusters_2 = dbscan_2.fit_predict(sorted_array_2)
    
        # Organize points into a dictionary based on their cluster
        cluster_dict_2 = {}
        for point, cluster_idx in zip(sorted_array_2, clusters_2):
            if cluster_idx != -1:  # Filter out noise points if needed
                cluster_dict_2.setdefault(cluster_idx, []).append(point.tolist())
    else:
        print("No blue points found in the image.")
        cluster_dict_2 = {}

    cluster_dict_2_proc = {}
    index = 0
    for key in cluster_dict_2:
        if(len(cluster_dict_2[key]) > 100):
            cluster_dict_2_proc[index] = cluster_dict_2[key]
            index += 1

    for cluster_idx in cluster_dict_2:
        cluster_dict_2[cluster_idx] = np.array(cluster_dict_2[cluster_idx])

    print(len(cluster_dict_2_proc))

    return roi, cluster_dict_1 , cluster_dict_2_proc


def inference_frame(inf_frame, mask_bound, hex_color_1, hex_color_2, cluster_dict_1, cluster_dict_2, roi, threshold = 40):
    
    x1, y1, x2, y2 = mask_bound
    inf_roi = inf_frame[y1:y2, x1:x2]
    
    # black
    bgr_color_1 = hex_to_bgr(hex_color_1) # "#C8CE7B"
    lower_bound_1 = np.array([max(0, bgr_color_1[0] - threshold), max(0, bgr_color_1[1] - threshold), max(0, bgr_color_1[2] - threshold)])
    upper_bound_1 = np.array([min(255, bgr_color_1[0] + threshold), min(255, bgr_color_1[1] + threshold), min(255, bgr_color_1[2] + threshold)])
    mask_1 = cv2.inRange(inf_roi, lower_bound_1, upper_bound_1)
    
    y_coord_1, x_coord_1 = np.where(mask_1 != 0)
    coord_array_1 = np.stack((y_coord_1, x_coord_1), axis=-1)
    inf_roi[mask_1 != 0] = [255, 0, 0]

    # white
    bgr_color_2 = hex_to_bgr(hex_color_2) # "#C8CE7B"
    lower_bound_2 = np.array([max(0, bgr_color_2[0] - threshold), max(0, bgr_color_2[1] - threshold), max(0, bgr_color_2[2] - threshold)])
    upper_bound_2 = np.array([min(255, bgr_color_2[0] + threshold), min(255, bgr_color_2[1] + threshold), min(255, bgr_color_2[2] + threshold)])
    mask_2 = cv2.inRange(inf_roi, lower_bound_2, upper_bound_2)
    
    y_coord_2, x_coord_2 = np.where(mask_2 != 0)
    coord_array_2 = np.stack((y_coord_2, x_coord_2), axis=-1)
    inf_roi[mask_2 != 0] = [255, 0, 255]

    all_errors_black = []
    error_keys_black = []
    for cluster in cluster_dict_1.values():
        error_count_1 = sum(1 for rows_ref, columns_ref in cluster 
                          if all(roi[rows_ref][columns_ref]) != all(inf_roi[rows_ref][columns_ref]))
        all_errors_black.append(error_count_1)

    for index, values in enumerate(all_errors_black):
        if(values > 20):
            error_keys_black.append(index)

    # White
    all_errors_white = []
    error_keys_white = []
    for cluster in cluster_dict_2.values():
        error_count_2 = sum(1 for rows_ref, columns_ref in cluster 
                          if all(roi[rows_ref][columns_ref]) != all(inf_roi[rows_ref][columns_ref]))
        all_errors_white.append(error_count_2)

    for index, values in enumerate(all_errors_white):
        if(values > 15):
            error_keys_white.append(index)

    print(all_errors_white)
        
    return inf_roi, error_keys_black, error_keys_white

In [35]:
cap =  cv2.VideoCapture(0, cv2.CAP_DSHOW)
ref_img = False 

while cap.isOpened():
    
    success, frame_img = cap.read()
    # frame = cv2.rotate(frame, cv2.ROTATE_180)

    if not success:
        print("Ignoring empty camera frame.")
        break

    if not ref_img:    
        cv2.imshow('Pressed Key Frame', frame_img)
        
        if cv2.waitKey(1) & 0xFF == ord('s'):
            mask_bound = (0, 265, 640, 452)
            roi, cluster_dict_1, cluster_dict_2 = reference_frame(frame_img, mask_bound, "#C8CE7B", "#699faf")
            # roi, cluster_dict_1 = reference_frame(frame_img, mask_bound, "#C8CE7B", "#699faf")
            ref_img = True
            frame_img = roi

    elif(ref_img):
        
        frame_roi, error_keys_black, error_keys_white = inference_frame(frame_img, mask_bound, "#C8CE7B", "#699faf", cluster_dict_1, cluster_dict_2, roi, threshold = 40)
        # frame_roi, error_keys_black = inference_frame(frame_img, mask_bound, "#C8CE7B", "#699faf", cluster_dict_1, " ",roi, threshold = 40)
        for keys in error_keys_black:
            for i in cluster_dict_1[keys]:
                rows, columns = i
                
                frame_roi[rows][columns][0] = 0
                frame_roi[rows][columns][1] = 0
                frame_roi[rows][columns][2] = 255

        for keys in error_keys_white:
            for i in cluster_dict_2[keys]:
                rows, columns = i
                
                frame_roi[rows][columns][0] = 0
                frame_roi[rows][columns][1] = 255 
                frame_roi[rows][columns][2] = 255
        
        cv2.imshow('Pressed Key Frame', frame_roi)

    cv2.waitKey(1)
    if cv2.getWindowProperty('Pressed Key Frame', cv2.WND_PROP_VISIBLE) < 1:
        break
    
cap.release()
cv2.destroyAllWindows()

5
[6, 10, 8, 1, 13]
[12, 17, 13, 2, 13]
[12, 14, 13, 2, 13]
[6, 8, 8, 2, 11]
[1, 3, 2, 2, 0]
[5, 7, 7, 2, 11]
[12, 17, 11, 2, 13]
[13, 16, 14, 3, 13]
[3, 7, 7, 2, 13]
[4, 7, 4, 0, 5]
[7, 11, 8, 2, 13]
[10, 16, 11, 2, 13]
[7, 14, 9, 2, 13]
[3, 1, 3, 1, 5]
[3, 6, 4, 2, 9]
[9, 14, 10, 2, 13]
[13, 9, 11, 2, 13]
[6, 14, 9, 3, 13]
[1, 6, 3, 2, 5]
[2, 5, 7, 2, 10]
[12, 14, 9, 2, 13]
[11, 15, 12, 2, 13]
[7, 11, 9, 2, 13]
[2, 7, 4, 1, 6]
[5, 10, 8, 2, 13]
[11, 16, 10, 2, 13]
[9, 15, 8, 3, 13]
[3, 10, 8, 2, 9]
[5, 7, 4, 2, 10]
[7, 11, 9, 2, 13]
[10, 12, 11, 2, 13]
[8, 13, 9, 3, 12]
[6, 9, 9, 1, 9]
[4, 8, 9, 3, 13]
[7, 13, 9, 2, 12]
[9, 13, 12, 2, 13]
[7, 14, 10, 2, 13]
[5, 12, 8, 2, 10]
[5, 8, 7, 3, 12]
[7, 11, 7, 2, 13]
[11, 12, 10, 2, 13]
[7, 13, 12, 3, 13]
[5, 7, 8, 2, 10]
[5, 10, 7, 2, 11]
[7, 13, 13, 2, 13]
[8, 14, 11, 2, 13]
[7, 10, 9, 3, 13]
[4, 9, 7, 3, 12]
[6, 9, 11, 2, 13]
[7, 13, 10, 3, 13]
[9, 13, 10, 2, 13]
[6, 15, 10, 2, 11]
[4, 8, 6, 1, 13]
[6, 9, 8, 2, 12]
[9, 14, 10, 3, 13]
[7, 

In [ ]:
# image = cv2.imread('frame_0.jpg')
# mask_bound = (0, 265, 640, 452)
# cluster_dict, roi = reference_frame(image, mask_bound, "#C8CE7B")

In [ ]:
# frame_img = cv2.imread('frame_3.jpg')
# frame_roi, error_keys = inference_frame(frame_img, mask_bound, "#C8CE7B", cluster_dict, roi, threshold = 40)